# Battery Model
## Problem 
- Build an optimisation model to charge/discharge the battery over the time period provided 
(2018-2020) in order to maximise profits. You may assume that the battery is a price-taker (ie. the 
actions of the battery have no impact on the market prices).
- You can trade in the 3 markets. 
- The battery can export any amount of power up to its maximum discharge rate for any 
duration of time, as long as it has sufficient energy stored to do so (And same with the charging -- needs storage)
- Markets 1 and 2 are traded at half-hourly time granularity, whereas Market 3 is traded at daily 
granularity 
- The battery cannot sell the same unit of power into multiple markets, but can divide its power 
across the markets (The energy you give or take to the markets must add up to the total energy difference)
- For the battery to participate in Markets 1 and 2, it must export/import a constant level of 
power for the full half-hour period 
For the battery to participate in Market 3, it must export/import a constant level of power for 
the full day 
## Inputs
- Parameters are stored in the "config.yml" file
- Price data is stored in csv file - "data/input_data/market12_data.csv" and "data/input_data/market3_data.csv"
- Also there is information about the total production and different types of electricity production
## Outputs
- [ ] Total profits (Revenue - capex - opex*time)
- [ ] Data - Total energy in, Total energy out + any other profits 
- [ ] Plots for all optimisation variables - NICE TO HAVE
- [ ] Short description
- [ ] Dependencies
## Process
- However, I have decided to create 5 tasks 
1. Decide energy in and out from Market 1 with no change in capacity 
2. Decide energy in and out from Market 1 with change in capacity
3. Decide energy in and out from Market 1 and 2 with change in capacity
4. Decide energy in and out from Market 1, 2 and 3 with change in capacity
5. Decide energy in and out from Market 1, 2 and 3 with change in capacity with difference in energy in and out efficiency. **(Explain later)**
6. Decide energy in and out from Market 1, 2 and 3 with change in capacity with weightings for different energy types.

## Variables
- $k$ is the sample number. Here it represents the number of half an hour steps the sample is from the start. 
- The prices $u_1[k]$, $u_2[k]$ and $u_3[k]$ represent the prices of each market at sample $k$. - market 1, 2 and 3 respectively
- The decision variables: 
- The energies into the battery at each sample ${x_{in}}_1[k]$, ${x_{in}}_2[k]$ and ${x_{in}}_3[k]$
- The energies into the battery at each sample ${x_{out}}_1[k]$, ${x_{out}}_2[k]$ and ${x_{out}}_3[k]$
- The net power into the battery at each sample (constant) $P_{in}[k]$
- The net energy of the battery at each sample $E_{net}[k]$
- The number of cycles at each sample $N_{cyc}[k]$
- The maximum capacity of the battery at each sample $C_{max}[k]$
## Model
$$P_{in}[k]=\frac{\Sigma^3_{i=1} {x_{in}}_i[k] - {x_{out}}_i[k]}{\Delta t}$$
$$E_{in}[k+1] = E_{in}[k] +(\Sigma^3_{i=1} {x_{in}}_i[k] - {x_{out}}_i[k])\times\mathtt{charging \ efficiency}$$
$$N_{cyc}[k+1]=N_{cyc}[k]+\frac{\Sigma^3_{i=1} {x_{in}}_i[k] - {x_{out}}_i[k]}{(2*C_{max}[k])}$$
- **ASSUMING MAX CAP DOESN'T CHANGE**
$$C_{max}[k+1]=C_{max}[k]$$

## Assumptions
- The battery can discharge any amount of energy up to it's current energy
- The battery can charge any amount of energy up to it's current remaining storage
- The power is constant throughout a sample
- The efficiency of the charging is 95% -- therefore the battery only gets 95% of what it buys
- The efficiency of the discharging is 95% -- therefore the battery only recieves 95% of the money that what the energy discharged cost
- **The battery maximum capacity does not change with time**
- **The battery can only use market 1**
## Initialisation
$$E_{net}[0]=0\mathtt{\ MW}\\
N_{cyc}[0]=0 \\ 
C_{max}=0\mathtt{\ MW}
$$

## Objective
- We want to maximise the profits by minimise the costs of the charging the batteries 
$$\begin{aligned}
\min_{{x_{in}}_1,{x_{out}}_1,{x_{in}}_2,{x_{out}}_2,{x_{in}}_3,{x_{out}}_3} \Sigma^3_{i=1} ({x_{in}}_i[k] - {x_{out}}_i[k])\times u_i[k]
\end{aligned}$$

## Constraints
- Additional to the model constaints
- Power is limited to the maximum charging and discharging rate 
$$\mathtt{max \ discharging \ rate} <= P_{in}[k] <= \mathtt{max \ discharging \ rate} $$
- The number of cycles is a parameter that must be exceeded
$$ 0 <= N_{cyc}[k] <= \mathtt{max \ cycles}$$
- The number of years is also limited
$$ 0 <= k / 31,536,000 <= \mathtt{max \ cycles}$$
- The energy is limited by its maximum capacity
$$ 0 <= E_{net}[k] <= C_{max}[k]$$
- The maximum capacity, although changes, never goes over the original max capacity
$$ 0 <= C_{max}[k] <= C_{max}[0]$$